In [1]:
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pickle
from ast import literal_eval
from textblob import TextBlob


In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
zf = zipfile.ZipFile(r'Times articles data\data\nyt-comments.zip') 
df_2018a = pd.read_csv(zf.open('CommentsApril2018.csv'))
df_2018m = pd.read_csv(zf.open('CommentsMarch2018.csv'))
df_2018f = pd.read_csv(zf.open('CommentsFeb2018.csv'))
df_2018j = pd.read_csv(zf.open('CommentsJan2018.csv'))

df_2017my = pd.read_csv(zf.open('CommentsMay2017.csv'))
df_2017a = pd.read_csv(zf.open('CommentsApril2017.csv'))
df_2017m = pd.read_csv(zf.open('CommentsMarch2017.csv'))
df_2017f = pd.read_csv(zf.open('CommentsFeb2017.csv'))
df_2017j = pd.read_csv(zf.open('CommentsJan2017.csv'))

c:\users\piotrwiercinski\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\piotrwiercinski\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (14,15,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\piotrwiercinski\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\piotrwiercinski\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarn

In [4]:
df_c2018 = pd.concat((pd.concat((df_2018a, df_2018m)), pd.concat((df_2018f, df_2018j))))

In [5]:
df_c2017 = pd.concat((pd.concat((pd.concat((df_2017my, df_2017a)), df_2017m)), pd.concat((df_2017f, df_2017j))))

In [6]:
df_c = pd.concat((df_c2018, df_c2017))

In [7]:
df_c.shape

(2176364, 34)

In [8]:
df_a = pd.read_csv(r'lda_data\lda_topics_df.csv',
                  index_col='index')

In [9]:
def load_obj(name ):
    with open(r'lda_data\\' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [10]:
topic_dict = load_obj('topics')

In [11]:
# df_a = df_a.sample(1000)  # UNCOMMENT THIS IF YOU WANT TO RUN THIS NOTEBOOK

# Running sentiment analysis on all comments takes about 4 hours, 1000 articles take less than 30mins on 7th gen i7 

In [12]:
df_a.head()

,articleID,keywords_x,text,corpus,lda_topics,abstract,articleWordCount,byline,documentType,headline,keywords_y,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
index,,,,,,,,,,,,,,,,,,,,
0,5adf6684068401528a2aa69b,"['offer', 'statement', 'cheerleading', 'suppor...",The N.F.L. has responded with a written statem...,"[(0, 1), (1, 1), (2, 3), (3, 5), (4, 2), (5, 3...","[(17, 0.08008672), (21, 0.022569574), (26, 0.1...",NaN,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,"['likely', 'epa', 'studies', 'effect', 'policy...",WASHINGTON — The Environmental Protection Agen...,"[(79, 1), (80, 1), (81, 1), (82, 5), (83, 1), ...","[(7, 0.0800965), (16, 0.44378182), (41, 0.0247...",NaN,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,"['probably', 'frequently', 'mr', 'asked', 'din...","If I get a reservation, will eating at Noma ma...","[(7, 1), (60, 1), (97, 2), (119, 1), (120, 1),...","[(0, 0.018988285), (2, 0.121632785), (9, 0.018...",NaN,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,"['iran', 'nuclear', 'constrain', 'mr', 'korea'...",WASHINGTON — President Trump signaled on Tuesd...,"[(42, 1), (50, 1), (97, 4), (99, 2), (116, 1),...","[(0, 0.020230627), (7, 0.06410627), (11, 0.438...",NaN,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,"['counts', 'women', 'murder', 'tuesdaythe', 'e...",TORONTO — The man identified as the van driver...,"[(38, 1), (76, 1), (97, 1), (113, 1), (116, 1)...","[(7, 0.086451806), (10, 0.021542551), (16, 0.0...",NaN,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [13]:
dict_assign = {}
for index, row in df_a.iterrows():
    dict_assign[index] = literal_eval(row.lda_topics)

In [14]:
dict_assign_topics = {}
dict_assign_weights = {}
for key, value in dict_assign.items():
    dict_assign_topics[key] = [x[0] for x in value]
    dict_assign_weights[key] = [x[1] for x in value]

In [15]:
dict_topic1 = {}
dict_topic2 = {}
for key in dict_assign_topics.keys():
    topic_weights = dict_assign_weights[key]
    topic_nums = dict_assign_topics[key]
    topic1_idx = topic_weights.index(max(topic_weights))
    dict_topic1[key] = topic_nums[topic1_idx]
    del topic_weights[topic1_idx]
    del topic_nums[topic1_idx]
    if topic_weights != []:
        topic2_idx = topic_weights.index(max(topic_weights))
        dict_topic2[key] = topic_nums[topic2_idx]
    

In [16]:
df_a['topic1'] = None
df_a['topic2'] = None

In [17]:
for index, row in df_a.iterrows():
    df_a.loc[index, 'topic1'] = dict_topic1[index]
    try:
        df_a.loc[index, 'topic2'] = dict_topic2[index]
    except:
        pass 

In [18]:
df_a.head()

,articleID,keywords_x,text,corpus,lda_topics,abstract,articleWordCount,byline,documentType,headline,keywords_y,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,topic1,topic2
index,,,,,,,,,,,,,,,,,,,,,,
0,5adf6684068401528a2aa69b,"['offer', 'statement', 'cheerleading', 'suppor...",The N.F.L. has responded with a written statem...,"[(0, 1), (1, 1), (2, 3), (3, 5), (4, 2), (5, 3...","[(17, 0.08008672), (21, 0.022569574), (26, 0.1...",NaN,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...,38,70
1,5adf653f068401528a2aa697,"['likely', 'epa', 'studies', 'effect', 'policy...",WASHINGTON — The Environmental Protection Agen...,"[(79, 1), (80, 1), (81, 1), (82, 5), (83, 1), ...","[(7, 0.0800965), (16, 0.44378182), (41, 0.0247...",NaN,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...,16,65
2,5adf4626068401528a2aa628,"['probably', 'frequently', 'mr', 'asked', 'din...","If I get a reservation, will eating at Noma ma...","[(7, 1), (60, 1), (97, 2), (119, 1), (120, 1),...","[(0, 0.018988285), (2, 0.121632785), (9, 0.018...",NaN,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...,27,63
3,5adf40d2068401528a2aa619,"['iran', 'nuclear', 'constrain', 'mr', 'korea'...",WASHINGTON — President Trump signaled on Tuesd...,"[(42, 1), (50, 1), (97, 4), (99, 2), (116, 1),...","[(0, 0.020230627), (7, 0.06410627), (11, 0.438...",NaN,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...,11,12
4,5adf3d64068401528a2aa60f,"['counts', 'women', 'murder', 'tuesdaythe', 'e...",TORONTO — The man identified as the van driver...,"[(38, 1), (76, 1), (97, 1), (113, 1), (116, 1)...","[(7, 0.086451806), (10, 0.021542551), (16, 0.0...",NaN,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...,65,63


In [19]:
df_a['topic1_words'] = df_a.topic1.apply(lambda x: topic_dict[x].split("\"")[1::2])

In [20]:
def try_get_words(x):
    try:
        return topic_dict[x].split("\"")[1::2]
    except:
        pass
    return None

In [21]:
df_a['topic2_words'] = df_a.topic2.apply(lambda x: try_get_words(x))

In [22]:
df_a.head()

,articleID,keywords_x,text,corpus,lda_topics,abstract,articleWordCount,byline,documentType,headline,keywords_y,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,topic1,topic2,topic1_words,topic2_words
index,,,,,,,,,,,,,,,,,,,,,,,,
0,5adf6684068401528a2aa69b,"['offer', 'statement', 'cheerleading', 'suppor...",The N.F.L. has responded with a written statem...,"[(0, 1), (1, 1), (2, 3), (3, 5), (4, 2), (5, 3...","[(17, 0.08008672), (21, 0.022569574), (26, 0.1...",NaN,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...,38,70,"[percent, year, job, economy, worker, economic...","[company, business, firm, kushner, sale, real_..."
1,5adf653f068401528a2aa697,"['likely', 'epa', 'studies', 'effect', 'policy...",WASHINGTON — The Environmental Protection Agen...,"[(79, 1), (80, 1), (81, 1), (82, 5), (83, 1), ...","[(7, 0.0800965), (16, 0.44378182), (41, 0.0247...",NaN,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...,16,65,"[study, percent, people, researcher, research,...","[energy, power, agency, turkey, environmental,..."
2,5adf4626068401528a2aa628,"['probably', 'frequently', 'mr', 'asked', 'din...","If I get a reservation, will eating at Noma ma...","[(7, 1), (60, 1), (97, 2), (119, 1), (120, 1),...","[(0, 0.018988285), (2, 0.121632785), (9, 0.018...",NaN,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...,27,63,"[food, restaurant, good, chicken, meal, meat, ...","[day, year, time, back, hour, old, first, home..."
3,5adf40d2068401528a2aa619,"['iran', 'nuclear', 'constrain', 'mr', 'korea'...",WASHINGTON — President Trump signaled on Tuesd...,"[(42, 1), (50, 1), (97, 4), (99, 2), (116, 1),...","[(0, 0.020230627), (7, 0.06410627), (11, 0.438...",NaN,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...,11,12,"[trump, president, mr, white_house, administra...","[north_korea, germany, britain, nuclear, europ..."
4,5adf3d64068401528a2aa60f,"['counts', 'women', 'murder', 'tuesdaythe', 'e...",TORONTO — The man identified as the van driver...,"[(38, 1), (76, 1), (97, 1), (113, 1), (116, 1)...","[(7, 0.086451806), (10, 0.021542551), (16, 0.0...",NaN,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...,65,63,"[energy, power, agency, turkey, environmental,...","[day, year, time, back, hour, old, first, home..."


In [23]:
df_c = df_c[df_c.articleID.isin(df_a.articleID)]

In [24]:
num_comments = df_c.groupby('articleID').size().to_frame()

In [25]:
num_comments.rename(columns={0: 'num_comments'}, inplace=True)

In [26]:
df_a.shape

(9329, 24)

In [27]:
df_a = df_a.merge(num_comments.reset_index(), on='articleID')

In [28]:
df_a.shape # some articles appear not to have any comments

(9329, 25)

In [29]:
df_a.head()

,articleID,keywords_x,text,corpus,lda_topics,abstract,articleWordCount,byline,documentType,headline,keywords_y,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,topic1,topic2,topic1_words,topic2_words,num_comments
0,5adf6684068401528a2aa69b,"['offer', 'statement', 'cheerleading', 'suppor...",The N.F.L. has responded with a written statem...,"[(0, 1), (1, 1), (2, 3), (3, 5), (4, 2), (5, 3...","[(17, 0.08008672), (21, 0.022569574), (26, 0.1...",NaN,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...,38,70,"[percent, year, job, economy, worker, economic...","[company, business, firm, kushner, sale, real_...",66
1,5adf653f068401528a2aa697,"['likely', 'epa', 'studies', 'effect', 'policy...",WASHINGTON — The Environmental Protection Agen...,"[(79, 1), (80, 1), (81, 1), (82, 5), (83, 1), ...","[(7, 0.0800965), (16, 0.44378182), (41, 0.0247...",NaN,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...,16,65,"[study, percent, people, researcher, research,...","[energy, power, agency, turkey, environmental,...",308
2,5adf4626068401528a2aa628,"['probably', 'frequently', 'mr', 'asked', 'din...","If I get a reservation, will eating at Noma ma...","[(7, 1), (60, 1), (97, 2), (119, 1), (120, 1),...","[(0, 0.018988285), (2, 0.121632785), (9, 0.018...",NaN,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...,27,63,"[food, restaurant, good, chicken, meal, meat, ...","[day, year, time, back, hour, old, first, home...",67
3,5adf40d2068401528a2aa619,"['iran', 'nuclear', 'constrain', 'mr', 'korea'...",WASHINGTON — President Trump signaled on Tuesd...,"[(42, 1), (50, 1), (97, 4), (99, 2), (116, 1),...","[(0, 0.020230627), (7, 0.06410627), (11, 0.438...",NaN,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...,11,12,"[trump, president, mr, white_house, administra...","[north_korea, germany, britain, nuclear, europ...",362
4,5adf3d64068401528a2aa60f,"['counts', 'women', 'murder', 'tuesdaythe', 'e...",TORONTO — The man identified as the van driver...,"[(38, 1), (76, 1), (97, 1), (113, 1), (116, 1)...","[(7, 0.086451806), (10, 0.021542551), (16, 0.0...",NaN,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...,65,63,"[energy, power, agency, turkey, environmental,...","[day, year, time, back, hour, old, first, home...",486


In [30]:
df_a.pubDate = pd.to_datetime(df_a.pubDate)

In [31]:
df_a['days_live'] =  pd.to_datetime('2018-05-03') - df_a.pubDate

In [32]:
df_a.days_live = df_a.days_live.dt.days

In [33]:
df_a['comments_per_day'] = df_a.num_comments / df_a.days_live

In [34]:
df_a.sort_values('comments_per_day', ascending=False, inplace=True)

In [35]:
df_c.head()

,approveDate,articleID,articleWordCount,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,inReplyTo,newDesk,parentID,parentUserDisplayName,permID,picURL,printPage,recommendations,recommendedFlag,replyCount,reportAbuseFlag,sectionName,sharing,status,timespeople,trusted,typeOfMaterial,updateDate,userDisplayName,userID,userLocation,userTitle,userURL
0,1524594282,5adf6684068401528a2aa69b,781.0,How could the league possibly refuse this offe...,26853969.0,26853969.0,<br/>,comment,1.524594e+09,1.0,0,0.0,Sports,0.0,NaN,26853969,http://graphics8.nytimes.com/images/apps/times...,0.0,0.0,NaN,0.0,NaN,Pro Football,0,approved,1.0,0.0,News,1524594282,Christopher Rillo,46566740.0,San Francisco,NaN,NaN
1,1524594252,5adf6684068401528a2aa69b,781.0,"So then the execs can be like ""yeah...we will ...",26853699.0,26853699.0,<br/>,comment,1.524593e+09,1.0,0,0.0,Sports,0.0,NaN,26853699,http://graphics8.nytimes.com/images/apps/times...,0.0,0.0,NaN,0.0,NaN,Pro Football,0,approved,1.0,0.0,News,1524594252,Matt Brand,64324866.0,"Williamsburg, Brooklyn",NaN,NaN
2,1524594250,5adf6684068401528a2aa69b,781.0,I would not want to play chess against these c...,26853677.0,26853677.0,<br/>,comment,1.524593e+09,1.0,0,0.0,Sports,0.0,NaN,26853677,http://graphics8.nytimes.com/images/apps/times...,0.0,0.0,NaN,0.0,NaN,Pro Football,0,approved,1.0,0.0,News,1524594250,Joseph,78105093.0,"Fayetteville, AR",NaN,NaN
3,1524593431,5adf6684068401528a2aa69b,781.0,Could the cheerleaders join the Actors' Equity...,26853784.0,26853784.0,<br/>,comment,1.524593e+09,1.0,0,0.0,Sports,0.0,NaN,26853784,http://graphics8.nytimes.com/images/apps/times...,0.0,3.0,NaN,0.0,NaN,Pro Football,0,approved,0.0,0.0,News,1524593431,Stephen,81939618.0,"Phoenix, AZ",NaN,NaN
4,1524595048,5adf653f068401528a2aa697,656.0,Seeking conclusions which support preconceived...,26854236.0,26854236.0,<br/>,comment,1.524595e+09,1.0,0,0.0,Climate,0.0,NaN,26854236,http://graphics8.nytimes.com/images/apps/times...,0.0,0.0,NaN,0.0,NaN,Unknown,1,approved,1.0,0.0,News,1524595048,Paul Zorsky,58642997.0,Texas,NaN,NaN


In [36]:
def get_comment_sentiment(articleID, df_c):
    com_text = '.'.join(df_c[df_c.articleID == articleID].commentBody.tolist())
    blob = TextBlob(com_text)
    sum_sentiment = 0
    len_blob_sentences = len(blob.sentences) + 0.001
    for sentence in blob.sentences:
        sum_sentiment += sentence.sentiment.polarity
    return sum_sentiment/len_blob_sentences
    

In [37]:
sen = get_comment_sentiment('5adef221068401528a2aa516', df_c)

In [38]:
sen

0.03013089835839926

In [39]:
def get_topic1_sentiment(articleID, df_a, df_c):
    com_text = '.'.join(df_c[df_c.articleID == articleID].commentBody.tolist())
    topic1_words = df_a[df_a.articleID == articleID].topic1_words
    relevant_sentences = [sentence + '.' for sentence in com_text.split('.') if any(x in topic1_words.values[0] for x in sentence.split())]
    relevant_text = ' '.join(relevant_sentences)
    blob = TextBlob(relevant_text)
    sum_sentiment = 0
    len_blob_sentences = len(blob.sentences) + 0.001
    for sentence in blob.sentences:
        sum_sentiment += sentence.sentiment.polarity
    return sum_sentiment/len_blob_sentences

In [40]:
t1 = get_topic1_sentiment('5adef221068401528a2aa516', df_a, df_c)

In [41]:
t1

0.0480058550012436

In [42]:
def get_topic2_sentiment(articleID, df_a, df_c):
    com_text = '.'.join(df_c[df_c.articleID == articleID].commentBody.tolist())
    topic2_words = df_a[df_a.articleID == articleID].topic2_words
    if topic2_words.values[0] == None:
        return None
    else:
        relevant_sentences = [sentence + '.' for sentence in com_text.split('.') if any(x in topic2_words.values[0] for x in sentence.split())]
        relevant_text = ' '.join(relevant_sentences)
        blob = TextBlob(relevant_text)
        sum_sentiment = 0
        len_blob_sentences = len(blob.sentences) + 0.001
        for sentence in blob.sentences:
            sum_sentiment += sentence.sentiment.polarity
        return sum_sentiment/len_blob_sentences

In [43]:
t2 = get_topic2_sentiment('5adef221068401528a2aa516', df_a, df_c)

In [44]:
t2

0.04164923862108632

In [45]:
df_a['article_sentiment'] = df_a.articleID.apply(lambda x: get_comment_sentiment(x, df_c))

In [46]:
df_a.article_sentiment.describe()

count    9329.000000
mean        0.086951
std         0.073897
min        -0.749251
25%         0.047032
50%         0.071315
75%         0.112264
max         0.924538
Name: article_sentiment, dtype: float64

In [47]:
df_a.head()

,articleID,keywords_x,text,corpus,lda_topics,abstract,articleWordCount,byline,documentType,headline,keywords_y,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,topic1,topic2,topic1_words,topic2_words,num_comments,days_live,comments_per_day,article_sentiment
1334,5ae7b8e9068401528a2ab8c7,"['twitter', 'moscow', 'ties', 'mr', 'russia', ...","WASHINGTON — Robert S. Mueller III, the specia...","[(39, 1), (97, 1), (105, 1), (114, 1), (116, 1...","[(6, 0.04533264), (7, 0.3590246), (11, 0.32174...",NaN,1452,By MICHAEL S. SCHMIDT,article,Questions for President Show Depth of Inquiry ...,['Russian Interference in 2016 US Elections an...,68,Washington,1,2018-05-01 00:46:23,Politics,The questions provide the most detailed look y...,The New York Times,News,https://www.nytimes.com/2018/04/30/us/politics...,7,11,"[mr, official, russian, russia, former, govern...","[trump, president, mr, white_house, administra...",2296,1,2296.000000,0.050904
1268,5ae5eb1c068401528a2ab52f,"['correspondents', 'women', 'kellyanne', 'conw...",WASHINGTON — The panna cotta had been served a...,"[(43, 4), (76, 3), (185, 1), (206, 1), (216, 1...","[(8, 0.01928886), (11, 0.18583831), (15, 0.255...",NaN,1302,By MICHAEL M. GRYNBAUM,article,Press Dinner’s Comedy Act Sets Off Furor,"['Wolf, Michelle', 'Talev, Margaret', 'Sanders...",68,Business,1,2018-04-29 15:56:08,Media,Remarks about Sarah Huckabee Sanders opened a ...,The New York Times,News,https://www.nytimes.com/2018/04/29/business/me...,15,11,"[inmate, chant, editorial, phoenix, scam, swan...","[trump, president, mr, white_house, administra...",1658,3,552.666667,0.032703
1321,5ae79ecd068401528a2ab893,"['working', 'trump', 'political', 'opinion', '...","So far, Donald Trump and his allies in Congres...","[(34, 2), (42, 1), (60, 1), (100, 1), (108, 1)...","[(11, 0.096760474), (30, 0.012253127), (38, 0....",NaN,894,By PAUL KRUGMAN,article,How’s That Tax Cut Working Out?,"['Corporate Taxes', 'United States Politics an...",68,OpEd,26,2018-04-30 22:55:07,Unknown,"Workers won’t see significant gains soon, if e...",The New York Times,Op-Ed,https://www.nytimes.com/2018/04/30/opinion/rep...,38,50,"[percent, year, job, economy, worker, economic...","[tax, people, plan, bill, health_care, year, h...",857,2,428.500000,0.084698
1333,5ae7b8d5068401528a2ab8c6,"['2017', 'clinton', 'mr', 'russia', 'trump', '...",What was your opinion of Mr. Comey during the ...,"[(29, 1), (40, 1), (42, 1), (43, 1), (60, 1), ...","[(7, 0.3048379), (11, 0.2062695), (28, 0.01303...",NaN,3074,By MATT APUZZO and MICHAEL S. SCHMIDT,article,"The Questions, With Context",['Russian Interference in 2016 US Elections an...,68,Washington,16,2018-05-01 00:46:10,Politics,The questions show the special counsel’s focus...,The New York Times,News,https://www.nytimes.com/2018/04/30/us/politics...,56,7,"[comey, investigation, memo, director, flynn, ...","[mr, official, russian, russia, former, govern...",364,1,364.000000,0.060461
1305,5ae6f8d0068401528a2ab6b6,"['officials', 'advisers', 'chinese', 'hard', '...",BEIJING — Staking an assertive negotiating sta...,"[(10, 1), (42, 2), (102, 2), (105, 1), (223, 2...","[(4, 0.07128326), (7, 0.11636938), (11, 0.2222...",NaN,1276,By KEITH BRADSHER,article,"Feeling It Has an Edge, China Sets a Hard Line...","['China', 'International Trade and World Marke...",68,Business,8,2018-04-30 11:06:50,Unknown,Beijing sees its economy as robust enough to d...,The New York Times,News,https://www.nytimes.com/2018/04/30/business/ch...,11,38,"[trump, president, mr, white_house, administra...","[percent, year, job, economy, worker, economic...",681,2,340.500000,0.068364


In [48]:
df_a['topic1_sentiment'] = df_a.articleID.apply(lambda x: get_topic1_sentiment(x, df_a, df_c))

In [49]:
def try_get_topic2_sentiment(x, df_a, df_c):
    try:
        return get_topic2_sentiment(x, df_a, df_c)
    except:
        pass

In [50]:
df_a['topic2_sentiment'] = df_a.articleID.apply(lambda x: get_topic2_sentiment(x, df_a, df_c))

In [51]:
df_a.head()

,articleID,keywords_x,text,corpus,lda_topics,abstract,articleWordCount,byline,documentType,headline,keywords_y,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,topic1,topic2,topic1_words,topic2_words,num_comments,days_live,comments_per_day,article_sentiment,topic1_sentiment,topic2_sentiment
1334,5ae7b8e9068401528a2ab8c7,"['twitter', 'moscow', 'ties', 'mr', 'russia', ...","WASHINGTON — Robert S. Mueller III, the specia...","[(39, 1), (97, 1), (105, 1), (114, 1), (116, 1...","[(6, 0.04533264), (7, 0.3590246), (11, 0.32174...",NaN,1452,By MICHAEL S. SCHMIDT,article,Questions for President Show Depth of Inquiry ...,['Russian Interference in 2016 US Elections an...,68,Washington,1,2018-05-01 00:46:23,Politics,The questions provide the most detailed look y...,The New York Times,News,https://www.nytimes.com/2018/04/30/us/politics...,7,11,"[mr, official, russian, russia, former, govern...","[trump, president, mr, white_house, administra...",2296,1,2296.000000,0.050904,0.039151,0.075692
1268,5ae5eb1c068401528a2ab52f,"['correspondents', 'women', 'kellyanne', 'conw...",WASHINGTON — The panna cotta had been served a...,"[(43, 4), (76, 3), (185, 1), (206, 1), (216, 1...","[(8, 0.01928886), (11, 0.18583831), (15, 0.255...",NaN,1302,By MICHAEL M. GRYNBAUM,article,Press Dinner’s Comedy Act Sets Off Furor,"['Wolf, Michelle', 'Talev, Margaret', 'Sanders...",68,Business,1,2018-04-29 15:56:08,Media,Remarks about Sarah Huckabee Sanders opened a ...,The New York Times,News,https://www.nytimes.com/2018/04/29/business/me...,15,11,"[inmate, chant, editorial, phoenix, scam, swan...","[trump, president, mr, white_house, administra...",1658,3,552.666667,0.032703,0.186857,0.005405
1321,5ae79ecd068401528a2ab893,"['working', 'trump', 'political', 'opinion', '...","So far, Donald Trump and his allies in Congres...","[(34, 2), (42, 1), (60, 1), (100, 1), (108, 1)...","[(11, 0.096760474), (30, 0.012253127), (38, 0....",NaN,894,By PAUL KRUGMAN,article,How’s That Tax Cut Working Out?,"['Corporate Taxes', 'United States Politics an...",68,OpEd,26,2018-04-30 22:55:07,Unknown,"Workers won’t see significant gains soon, if e...",The New York Times,Op-Ed,https://www.nytimes.com/2018/04/30/opinion/rep...,38,50,"[percent, year, job, economy, worker, economic...","[tax, people, plan, bill, health_care, year, h...",857,2,428.500000,0.084698,0.114945,0.107638
1333,5ae7b8d5068401528a2ab8c6,"['2017', 'clinton', 'mr', 'russia', 'trump', '...",What was your opinion of Mr. Comey during the ...,"[(29, 1), (40, 1), (42, 1), (43, 1), (60, 1), ...","[(7, 0.3048379), (11, 0.2062695), (28, 0.01303...",NaN,3074,By MATT APUZZO and MICHAEL S. SCHMIDT,article,"The Questions, With Context",['Russian Interference in 2016 US Elections an...,68,Washington,16,2018-05-01 00:46:10,Politics,The questions show the special counsel’s focus...,The New York Times,News,https://www.nytimes.com/2018/04/30/us/politics...,56,7,"[comey, investigation, memo, director, flynn, ...","[mr, official, russian, russia, former, govern...",364,1,364.000000,0.060461,0.044602,0.029098
1305,5ae6f8d0068401528a2ab6b6,"['officials', 'advisers', 'chinese', 'hard', '...",BEIJING — Staking an assertive negotiating sta...,"[(10, 1), (42, 2), (102, 2), (105, 1), (223, 2...","[(4, 0.07128326), (7, 0.11636938), (11, 0.2222...",NaN,1276,By KEITH BRADSHER,article,"Feeling It Has an Edge, China Sets a Hard Line...","['China', 'International Trade and World Marke...",68,Business,8,2018-04-30 11:06:50,Unknown,Beijing sees its economy as robust enough to d...,The New York Times,News,https://www.nytimes.com/2018/04/30/business/ch...,11,38,"[trump, president, mr, white_house, administra...","[percent, year, job, economy, worker, economic...",681,2,340.500000,0.068364,0.046785,0.097829


In [52]:
df_a.topic1_sentiment.describe()

count    9329.000000
mean        0.097757
std         0.101014
min        -0.499500
25%         0.039992
50%         0.086001
75%         0.137427
max         0.999001
Name: topic1_sentiment, dtype: float64

In [53]:
df_a.topic2_sentiment.describe()

count    9283.000000
mean        0.096005
std         0.107655
min        -0.599401
25%         0.032277
50%         0.085504
75%         0.138822
max         0.999001
Name: topic2_sentiment, dtype: float64

In [54]:
df_a.to_csv(r'lda_data\comment_analysis.csv', index_label='index')